# Bagging

In [1]:
import numpy as np
import matplotlib.pyplot as plt

The boostrapping has the following steps:

1. Sample $m$ times **with replacement** from the original training data
2. Repeat $B$ times to generate $B$ "boostrapped" training datasets $D_1, D_2, \cdots, D_B$
3. Train $B$ trees using the training datasets $D_1, D_2, \cdots, D_B$ 

Boostrapping the data plus performing some sort of aggregation (averaging or majority votes) is called **boostrap aggregation** or **bagging**.

*Example*:

Assume that we have a training set where $m=4$, and $n=2$:

$$D = {(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4)}$$

We generate, say, $B = 3$ datasets by boostrapping:

$$D_1 = {(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_3, y_3)}$$
$$D_2 = {(x_1, y_1), (x_4, y_4), (x_4, y_4), (x_3, y_3)}$$
$$D_3 = {(x_1, y_1), (x_1, y_1), (x_2, y_2), (x_2, y_2)}$$

We can then train 3 trees.

Note: When sampling is performed **without** replacement, it is called **pasting**.

## 1. Scratch

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

In [3]:
X_train.shape

(105, 4)

In [4]:
#import
from sklearn.tree import DecisionTreeClassifier
import random
from scipy import stats
from sklearn.metrics import classification_report

#define some parameters
B = 5
m, n = X_train.shape
bootstrap_ratio = 1
tree_params = {'max_depth': 2, 'criterion': 'gini', 'min_samples_split': 5}
models = [DecisionTreeClassifier(**tree_params) for _ in range(B)]

sample_size = int(bootstrap_ratio * len(X_train))

xsamples = np.zeros((B, sample_size, n))
ysamples = np.zeros((B, sample_size))

#define bootstrapping
for i in range(B):
    for j in range(sample_size):
        idx = random.randrange(m) #<----with replacement
        xsamples[i, j, :] = X_train[idx]
        ysamples[i, j]    = y_train[idx]

#fit each tree
for i, model in enumerate(models):
    _X = xsamples[i, :]
    _y = ysamples[i, :]
    model.fit(_X, _y)
    
#make predictions - take majority - mode
predictions = np.zeros((B, X_test.shape[0]))
for i, model in enumerate(models):
    yhat = model.predict(X_test)
    predictions[i, :] = yhat

yhat = stats.mode(predictions)[0][0] 

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.85      0.92        13
           2       0.87      1.00      0.93        13

    accuracy                           0.96        45
   macro avg       0.96      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45



/var/folders/1v/1h7r513n71j7569x87spb4b40000gn/T/ipykernel_30921/2126721457.py:38: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  yhat = stats.mode(predictions)[0][0]


In [ ]:
#ensemble - using multiple classifiers together to make a joint decision